# Fine Tuning all Models for Edge-AI Compatibility

## Relevant Dependencies and Loading pretrained weights (832x832)

In [ ]:
%pip install -q ultralytics -U
%pip install -q depthai==3.0.0rc2 -U
%pip install -q depthai-nodes==0.3.0 -U

In [ ]:
try:
    from google.colab import drive as _colab_drive
    try:
        from google.colab import runtime as _colab_runtime
    except Exception:
        _colab_runtime = None
    IN_COLAB = True
except Exception:
    _colab_drive = None
    _colab_runtime = None
    IN_COLAB = False

# removed drive.mount for non-Colab

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO

model_yolo_12_n = YOLO("./YOLO_Runs/segment/y12nseg_scratch_832_20251003_2047/weights/best.pt")
model_yolo_12_s = YOLO("./YOLO_Runs/segment/y12sseg_scratch_832_20251004_1104/weights/best.pt")
model_yolo_11_n = YOLO("./YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt")
model_yolo_11_s = YOLO("./YOLO_Runs/segment/y11sseg_832_b32_pat60_20251001_2215/weights/best.pt")
model_yolo_8_n = YOLO("./YOLO_Runs/segment/y8nseg_832_b32_pat60_20251009_2123/weights/best.pt")
model_yolo_8_s = YOLO("./YOLO_Runs/segment/y8sseg_832_b32_pat60_20251009_2300/weights/best.pt")

## Fine Tuning

In [ ]:
try:
    from google.colab import drive as _colab_drive
    try:
        from google.colab import runtime as _colab_runtime
    except Exception:
        _colab_runtime = None
    IN_COLAB = True
except Exception:
    _colab_drive = None
    _colab_runtime = None
    IN_COLAB = False

from ultralytics import YOLO
from datetime import datetime


data_yaml = "./RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    task="segment",
    data=data_yaml,
    imgsz=640,
    batch=32, workers=2,
    epochs=50, patience=15,
    lr0=0.003,
    warmup_epochs=2,
    pretrained=True,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentationen für Fine Tuning
    fliplr=0.5, flipud=0.10, degrees=10.0, shear=2.0, perspective=0.000,
    scale=0.50, translate=0.10,
    hsv_h=0.02, hsv_s=0.70, hsv_v=0.40,
    mosaic=0.6, close_mosaic=20, mixup=0.05, copy_paste=0.30,
    erasing=0.2
)

# YOLOv11n-seg
run_11n = f"y11n_seg_ft"
model_yolo_11_n.train(project="./YOLO_Runs/segment", name=run_11n, **common)

# YOLOv11s-seg
run_11s = f"y11s_seg_ft"
model_yolo_11_s.train(project="./YOLO_Runs/segment", name=run_11s, **common)

# YOLOv8n-seg
run_8n = f"y8n_seg_ft"
model_yolo_8_n.train(project="./YOLO_Runs/segment", name=run_8n, **common)

# YOLOv8s-seg
run_8s = f"y8s_seg_ft"
model_yolo_8_s.train(project="./YOLO_Runs/segment", name=run_8s, **common)

# YOLOv12n-seg
run_12n = f"y12n_seg_ft"
model_yolo_12_n.train(project="./YOLO_Runs/segment", name=run_12n, **common)

# YOLOv12s-seg
run_12s = f"y12s_seg_ft"
model_yolo_12_s.train(project="./YOLO_Runs/segment", name=run_12s, **common)

_colab_runtime.unassign()

Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=/content/drive/MyDrive/YOLO_Runs/segment/y11nseg_832_b32_pat60_20251001_16492/weights/best.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=y11n_seg_ft, nbs=64, n

## Convertion for further Purposes

Hier können die YOLOv12 Modelle nicht verwendet werden, da die Modelle nicht kompatibel zu RVC2 sind


In [ ]:
%pip install -q ultralytics -U
%pip uninstall -y numpy
%pip install -U "numpy<2.0" "luxonis-ml[data]" modelconv==0.4.0
%pip install -U depthai==3.0.0rc2 depthai-nodes==0.3.0
# Runtime hart neu starten, damit die neuen Wheels geladen werden:
import os, sys; os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 94.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 88.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of luxonis-ml[data] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86


In [ ]:
import shutil

shutil.copy("./YOLO_Runs/segment/y11n_seg_ft/weights/best.pt", "yolo11n_seg_trained.pt")
shutil.copy("./YOLO_Runs/segment/y11s_seg_ft/weights/best.pt", "yolo11s_seg_trained.pt")
shutil.copy("./YOLO_Runs/segment/y8n_seg_ft/weights/best.pt", "yolo8n_seg_trained.pt")
shutil.copy("./YOLO_Runs/segment/y8s_seg_ft/weights/best.pt", "yolo8s_seg_trained.pt")

'yolo8s_seg_trained.pt'

In [ ]:
HUBAI_API_KEY = "KEY"

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo11n_seg_trained.pt",
    name="YOLO11 Nano 640",
    description_short="x",
    yolo_version="yolov11",
    yolo_input_shape="640 640",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=530198;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=708323;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model variant 'YOLO11 Nano 640' created with ID '19ad3dde-6cba-4d0d-bf30-96dc82884354'

Model instance 'YOLO11 Nano 640 base instance' created with ID '446c8212-4a68-4bfb-bc16-6da80f1c53c7'

File '/content/yolo11n_seg_trained.pt' uploaded to model instance '446c8212-4a68-4bfb-bc16-6da80f1c53c7'

Model instance 'YOLO11 Nano 640 exported to Target.RVC2' created for RVC2 export with ID 
'2d99b019-88ed-45c0-bc78-93ae71f2591a'

Output()

Output()

Downloaded 'yolo11n_seg_trained.rvc2.tar.xz'

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo11s_seg_trained.pt",
    name="YOLO11s Seg 640",
    description_short="RVC2 640x640",
    yolo_version="yolov11",
    yolo_input_shape="640 640",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False,
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=786093;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=881456;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model variant 'YOLO11s Seg 640' created with ID '4a397fdb-fbe9-4129-9a83-fef30f9a7e8e'

Model instance 'YOLO11s Seg 640 base instance' created with ID '0860084e-2f36-4a9d-9219-851119951ff0'

File '/content/yolo11s_seg_trained.pt' uploaded to model instance '0860084e-2f36-4a9d-9219-851119951ff0'

Model instance 'YOLO11s Seg 640 exported to Target.RVC2' created for RVC2 export with ID 
'47b73c0d-2f7b-4222-ab81-f2cacb8a3e91'

Output()

Output()

Downloaded 'yolo11s_seg_trained.rvc2.tar.xz'

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo8n_seg_trained.pt",
    name="YOLOv8n Seg 640",
    description_short="RVC2 640x640",
    yolo_version="yolov8",
    yolo_input_shape="640 640",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=571165;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=26348;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model 'YOLOv8n Seg 640' created with ID '8748612d-7835-4be7-97fb-f5fe2b6e8878'

Model variant 'YOLOv8n Seg 640' created with ID '4bbec6ca-0923-4e6a-ae39-ab7b23ff08f3'

Model instance 'YOLOv8n Seg 640 base instance' created with ID '1f819310-5220-4c7b-9bb9-d7a59b15f871'

File '/content/yolo8n_seg_trained.pt' uploaded to model instance '1f819310-5220-4c7b-9bb9-d7a59b15f871'

Model instance 'YOLOv8n Seg 640 exported to Target.RVC2' created for RVC2 export with ID 
'5875822a-8447-42a2-a7b5-3a6b96663cdd'

Output()

Output()

Downloaded 'yolo8n_seg_trained.rvc2.tar.xz'

In [ ]:
from modelconverter import convert

converted_model = convert.RVC2(
    api_key=HUBAI_API_KEY,
    path="/content/yolo8s_seg_trained.pt",
    name="YOLOv8s Seg 640",
    description_short="RVC2 640x640",
    yolo_version="yolov8",
    yolo_input_shape="640 640",
    yolo_class_names=["Kugellager", "Motor", "Schraubenzieher"],
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)

INFO     Detected PyTorch model. Only YOLO models are supported.                                      ]8;id=576434;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py\config.py]8;;\:]8;id=728251;file:///usr/local/lib/python3.12/dist-packages/modelconverter/utils/config.py#308\308]8;;\

Model 'YOLOv8s Seg 640' created with ID 'f4697506-592e-4874-b0ca-2f17d820f15c'

Model variant 'YOLOv8s Seg 640' created with ID '8457ac60-c35d-46f1-9dc5-d0df8dee2061'

Model instance 'YOLOv8s Seg 640 base instance' created with ID '1541e49d-55d7-4ea9-a250-b75f5efbb8aa'

File '/content/yolo8s_seg_trained.pt' uploaded to model instance '1541e49d-55d7-4ea9-a250-b75f5efbb8aa'

Model instance 'YOLOv8s Seg 640 exported to Target.RVC2' created for RVC2 export with ID 
'4eb8c663-c5da-4a63-9fb4-cf7988a7c279'

Output()

Output()

Downloaded 'yolo8s_seg_trained.rvc2.tar.xz'